In [1]:
import numpy as np
import xarray as xr
from climlab_rrtmg import rrtmg_lw, rrtmg_sw

## Get the input data

In [2]:
class gcm_data:
    def __init__(self,ncdf_name,t,j,i):
        # t: time indice; j: lat indice; i: lon indice
        #===================paramters for the conversion from mass to volume mixing ratio===========================
        amdw = 1.607793  # Molecular weight of dry air / water vapor
        amdo = 0.603428  # Molecular weight of dry air / ozone
        amdc = 0.658114  # Molecular weight of dry air / carbon dioxide
        amdm = 1.805423  # Molecular weight of dry air / methane
        amdn = 0.658090  # Molecular weight of dry air / nitrous oxide
        amdo2 = 0.905140 # Molecular weight of dry air / oxygen
        amdc1 = 0.210852 # Molecular weight of dry air / CFC11
        amdc2 = 0.239546 # Molecular weight of dry air / CFC12
        #=======================lat and lon==================================================================================
        self.lat=ncdf_name.lat[j]
        self.lon=ncdf_name.lon[i]
        #========================give all the inputs=================================================================
        self.nlay = ncdf_name.lev.size

        # interface pressure
        pint=ncdf_name.PNM_0e_to_360e_10n_to_40n[t,:,j,i] #dynes/cm2
        plev=np.array(pint[::-1]/1000) #hPa, from surface to top
        self.plev = plev[np.newaxis, ...]
         
        # midpoint pressure
        pmid=ncdf_name.PBR_0e_to_360e_10n_to_40n[t,:,j,i] #dynes/cm2
        play=np.array(pmid[::-1]/1000) #hPa, from surface to top
        self.play = play[np.newaxis, ...]

        # hybrid level at interfaces
        ilev  =ncdf_name.ilev # from top to surface
        lev_i=np.array(ilev) # from top to surface
        self.ilev = lev_i

        # hybrid level at midpoints
        lev  =ncdf_name.lev # from top to surface
        lev_m=np.array(lev) # from top to surface
        self.mlev = lev_m

        # surface temperature
        tint=ncdf_name.TINT_0e_to_360e_10n_to_40n[t,:,j,i] #K
        self.tsfc =np.array(tint[-1])

        # interface temperature
        tlev=np.array(tint[::-1])      #K
        self.tlev= tlev[np.newaxis, ...]

        # midpoint temperature
        tmid=ncdf_name.TMID_0e_to_360e_10n_to_40n[t,:,j,i] #K
        tlay=np.array(tmid[::-1])
        self.tlay= tlay[np.newaxis, ...]

        # specific humidity
        sp_hum=ncdf_name.SP_HUM_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio, kg/kg
        # convert the mass mixing ratio to volume mixing ratio
        h2ovmr = np.array(sp_hum[::-1]) * amdw
        self.h2ovmr = h2ovmr[np.newaxis, ...]

        # O3
        O3=ncdf_name.O3_rad_0e_to_360e_10n_to_40n[t,:,j,i] # mass mixing ratio, kg/kg
        O3vmr = np.array(O3[::-1]) * amdo
        self.o3vmr = O3vmr[np.newaxis, ...]

        # CO2
#         co2vmr=ncdf_name.co2vmr[t]
#         self.co2vmr=np.array(co2vmr)* np.ones_like(self.play)
        
        co2=ncdf_name.CO2_rad_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio
        co2vmr=np.array(co2[::-1])*amdc
        self.co2vmr = co2vmr[np.newaxis, ...]

        # CH4
        ch4=ncdf_name.CH4_rad_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio
        ch4vmr=np.array(ch4[::-1])*amdm
        self.ch4vmr = ch4vmr[np.newaxis, ...]

        # N2O
        n2o=ncdf_name.N2O_rad_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio
        n2ovmr=np.array(n2o[::-1])*amdn
        self.n2ovmr = n2ovmr[np.newaxis, ...]

        # CFC11
        cfc11=ncdf_name.CFC11_rad_0e_to_360e_10n_to_40n[t,:,j,i]
        cfc11vmr=np.array(cfc11[::-1])*amdc1
        self.cfc11vmr = cfc11vmr[np.newaxis, ...]

        # CFC12
        cfc12=ncdf_name.CFC12_rad_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio
        cfc12vmr=np.array(cfc12[::-1])*amdc2
        self.cfc12vmr = cfc12vmr[np.newaxis, ...]

        # O2
        o2=ncdf_name.O2_rad_0e_to_360e_10n_to_40n[t,:,j,i] #mass mixing ratio
        o2vmr=np.array(o2[::-1])*amdo2
        self.o2vmr = o2vmr[np.newaxis, ...]

        # cloud fraction
        cld=ncdf_name.CLD_rad_0e_to_360e_10n_to_40n[t,:,j,i] # no units
        cld=np.array(cld[::-1])
        self.cldfrac = cld[np.newaxis, ...]

        # In-cloud liquid water path (?) # What is the unit here?
        clwp=ncdf_name.CLIQWP_rad_0e_to_360e_10n_to_40n[t,:,j,i] # g/m2
        clwp=np.array(clwp[::-1])
        self.clwp = clwp[np.newaxis, ...]
        
        # In-cloud ice water path (?) # What is the unit here?
        ciwp=ncdf_name.CICEWP_rad_0e_to_360e_10n_to_40n[t,:,j,i]  # g/m2
        ciwp=np.array(ciwp[::-1])
        self.ciwp = ciwp[np.newaxis, ...]
                
        # Cloud water drop effective radius (microns)
        relq=ncdf_name.REL_rad_0e_to_360e_10n_to_40n[t,:,j,i]
        relq=np.array(relq[::-1])
        self.relq = relq[np.newaxis, ...]
        
        # Cloud ice particle effective size (microns)
        reic=ncdf_name.REI_rad_0e_to_360e_10n_to_40n[t,:,j,i]
        reic=np.array(reic[::-1])
        self.reic = reic[np.newaxis, ...]
        
        self.cfc22vmr = 0. * np.ones_like(self.play)
        self.ccl4vmr = 0. * np.ones_like(self.play)

        # Longwave diffuse albedo
        aldif=ncdf_name.ALDIF_rad_0e_to_360e_10n_to_40n[t,j,i]
        self.aldif=np.array(aldif)

        # Longwave direct albedo
        aldir=ncdf_name.ALDIR_rad_0e_to_360e_10n_to_40n[t,j,i]
        self.aldir=np.array(aldir)

        # Shortwave diffuse albedo
        asdif=ncdf_name.ASDIF_rad_0e_to_360e_10n_to_40n[t,j,i]
        self.asdif=np.array(asdif)

        # Shortwave direct albedo
        asdir=ncdf_name.ASDIR_rad_0e_to_360e_10n_to_40n[t,j,i]
        self.asdir=np.array(asdir)

        # cosine of the solar zenith angle
        coszen=ncdf_name.COSZRS_0e_to_360e_10n_to_40n[t,j,i]
        self.coszen=np.array(coszen)

        # eccentricity factor
        eccf=ncdf_name.ECCF_0e_to_360e_10n_to_40n[t,j,i]
        self.eccf=np.array(eccf)

In [2]:
class gcm_daily_data:
    def __init__(self,ncdf_name,j,i):
        # t: time indice; j: lat indice; i: lon indice
        #===================paramters for the conversion from mass to volume mixing ratio===========================
        amdw = 1.607793  # Molecular weight of dry air / water vapor
        amdo = 0.603428  # Molecular weight of dry air / ozone
        amdc = 0.658114  # Molecular weight of dry air / carbon dioxide
        amdm = 1.805423  # Molecular weight of dry air / methane
        amdn = 0.658090  # Molecular weight of dry air / nitrous oxide
        amdo2 = 0.905140 # Molecular weight of dry air / oxygen
        amdc1 = 0.210852 # Molecular weight of dry air / CFC11
        amdc2 = 0.239546 # Molecular weight of dry air / CFC12
        #=======================lat and lon==================================================================================
        self.lat=ncdf_name.lat[j]
        self.lon=ncdf_name.lon[i]
        #========================give all the inputs=================================================================
        self.nlay = ncdf_name.lev.size

        # interface pressure
        pint=ncdf_name.PNM[:,:,j,i].mean(dim='time') #dynes/cm2
        plev=np.array(pint[::-1]/1000) #hPa, from surface to top
        self.plev = plev[np.newaxis, ...]
         
        # midpoint pressure
        pmid=ncdf_name.PBR[:,:,j,i].mean(dim='time') #dynes/cm2
        play=np.array(pmid[::-1]/1000) #hPa, from surface to top
        self.play = play[np.newaxis, ...]

        # hybrid level at interfaces
        ilev  =ncdf_name.ilev # from top to surface
        lev_i=np.array(ilev) # from top to surface
        self.ilev = lev_i

        # hybrid level at midpoints
        lev  =ncdf_name.lev # from top to surface
        lev_m=np.array(lev) # from top to surface
        self.mlev = lev_m

        # surface temperature
        tint=ncdf_name.TINT[:,:,j,i].mean(dim='time') #K
        self.tsfc =np.array(tint[-1])

        # interface temperature
        tlev=np.array(tint[::-1])      #K
        self.tlev= tlev[np.newaxis, ...]

        # midpoint temperature
        tmid=ncdf_name.TMID[:,:,j,i].mean(dim='time') #K
        tlay=np.array(tmid[::-1])
        self.tlay= tlay[np.newaxis, ...]

        # specific humidity
        sp_hum=ncdf_name.SP_HUM[:,:,j,i].mean(dim='time') #mass mixing ratio, kg/kg
        # convert the mass mixing ratio to volume mixing ratio
        h2ovmr = np.array(sp_hum[::-1]) * amdw
        self.h2ovmr = h2ovmr[np.newaxis, ...]

        # O3
        O3=ncdf_name.O3_rad[:,:,j,i].mean(dim='time') # mass mixing ratio, kg/kg
        O3vmr = np.array(O3[::-1]) * amdo
        self.o3vmr = O3vmr[np.newaxis, ...]

        # CO2
#         co2vmr=ncdf_name.co2vmr[t]
#         self.co2vmr=np.array(co2vmr)* np.ones_like(self.play)
        
        co2=ncdf_name.CO2_rad[:,:,j,i].mean(dim='time') #mass mixing ratio
        co2vmr=np.array(co2[::-1])*amdc
        self.co2vmr = co2vmr[np.newaxis, ...]

        # CH4
        ch4=ncdf_name.CH4_rad[:,:,j,i].mean(dim='time') #mass mixing ratio
        ch4vmr=np.array(ch4[::-1])*amdm
        self.ch4vmr = ch4vmr[np.newaxis, ...]

        # N2O
        n2o=ncdf_name.N2O_rad[:,:,j,i].mean(dim='time') #mass mixing ratio
        n2ovmr=np.array(n2o[::-1])*amdn
        self.n2ovmr = n2ovmr[np.newaxis, ...]

        # CFC11
        cfc11=ncdf_name.CFC11_rad[:,:,j,i].mean(dim='time')
        cfc11vmr=np.array(cfc11[::-1])*amdc1
        self.cfc11vmr = cfc11vmr[np.newaxis, ...]

        # CFC12
        cfc12=ncdf_name.CFC12_rad[:,:,j,i].mean(dim='time') #mass mixing ratio
        cfc12vmr=np.array(cfc12[::-1])*amdc2
        self.cfc12vmr = cfc12vmr[np.newaxis, ...]

        # O2
        o2=ncdf_name.O2_rad[:,:,j,i].mean(dim='time') #mass mixing ratio
        o2vmr=np.array(o2[::-1])*amdo2
        self.o2vmr = o2vmr[np.newaxis, ...]

        # cloud fraction
        cld=ncdf_name.CLD_rad[:,:,j,i].mean(dim='time') # no units
        cld=np.array(cld[::-1])
        self.cldfrac = cld[np.newaxis, ...]

        # In-cloud liquid water path (?) # What is the unit here?
        clwp=ncdf_name.CLIQWP_rad[:,:,j,i].mean(dim='time') # g/m2
        clwp=np.array(clwp[::-1])
        self.clwp = clwp[np.newaxis, ...]
        
        # In-cloud ice water path (?) # What is the unit here?
        ciwp=ncdf_name.CICEWP_rad[:,:,j,i].mean(dim='time')  # g/m2
        ciwp=np.array(ciwp[::-1])
        self.ciwp = ciwp[np.newaxis, ...]
                
        # Cloud water drop effective radius (microns)
        relq=ncdf_name.REL_rad[:,:,j,i].mean(dim='time')
        relq=np.array(relq[::-1])
        self.relq = relq[np.newaxis, ...]
        
        # Cloud ice particle effective size (microns)
        reic=ncdf_name.REI_rad[:,:,j,i].mean(dim='time')
        reic=np.array(reic[::-1])
        self.reic = reic[np.newaxis, ...]
        
        self.cfc22vmr = 0. * np.ones_like(self.play)
        self.ccl4vmr = 0. * np.ones_like(self.play)

        # Longwave diffuse albedo
        aldif=ncdf_name.ALDIF_rad[:,j,i].mean(dim='time')
        self.aldif=np.array(aldif)

        # Longwave direct albedo
        aldir=ncdf_name.ALDIR_rad[:,j,i].mean(dim='time')
        self.aldir=np.array(aldir)

        # Shortwave diffuse albedo
        asdif=ncdf_name.ASDIF_rad[:,j,i].mean(dim='time')
        self.asdif=np.array(asdif)

        # Shortwave direct albedo
        asdir=ncdf_name.ASDIR_rad[:,j,i].mean(dim='time')
        self.asdir=np.array(asdir)

        # cosine of the solar zenith angle
        coszen=ncdf_name.COSZRS[:,j,i].mean(dim='time')
        self.coszen=np.array(coszen)

        # eccentricity factor
        eccf=ncdf_name.ECCF[:,j,i].mean(dim='time')
        self.eccf=np.array(eccf)

In [7]:
def offline(nlay,plev,play,tsfc,tlev,tlay,h2ovmr,o3vmr,co2vmr,ch4vmr,n2ovmr,cfc11vmr,cfc12vmr,o2vmr,cfc22vmr,ccl4vmr,\
           cldfrac,ciwp, clwp, reic, relq, asdir, asdif, aldir, aldif,coszen,eccf,mlev,ilev):
    
    # Specific heat at constant pressure
    cp = 1004.
    ncol = 1

    #====================================Do the longwave calculation==============================================
    # nbndlw and ngptlw
    nbndlw=rrtmg_lw.parrrtm.nbndlw
    ngptlw=rrtmg_lw.parrrtm.ngptlw 
    
    ## First, initialize the data
    rrtmg_lw.climlab_rrtmg_lw_ini(cp)

    ## Lots of RRTMG parameters
    icld = 1    # Cloud overlap method, 0: Clear only, 1: Random, 2,  Maximum/random] 3: Maximum
    irng = 1  # more monte carlo stuff
    idrv = 0  # whether to also calculate the derivative of flux with respect to surface temp
    permuteseed = 300
    inflglw  = 2
    # Set all these variables to 0, same as CCM3:
    # https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/rrtmg_lw_v4.85/gcm_model/src/rrtmg_lw_cldprmc.f90
    iceflglw = 0
    liqflglw = 0

   
    #  These arrays have an extra dimension for number of bands
    # in-cloud optical depth [nbndlw,ncol,nlay]
    tauc = 0. * np.ones_like(play)
    #  broadcast to get [nbndlw,ncol,nlay]
    tauc = tauc * np.ones([nbndlw,ncol,nlay])
    # Aerosol optical depth at mid-point of LW spectral bands [ncol,nlay,nbndlw]
    tauaer = 0. * np.ones_like(play)
    #  broadcast and transpose to get [ncol,nlay,nbndlw]
    tauaer = np.transpose(tauaer * np.ones([nbndlw,ncol,nlay]), (1,2,0))

    # surface emissivity
    emis = 1. * np.ones((ncol,nbndlw))

#     # Clear-sky only first
#     cldfmcl = np.zeros((ngptlw,ncol,nlay))
#     ciwpmcl = np.zeros((ngptlw,ncol,nlay))
#     clwpmcl = np.zeros((ngptlw,ncol,nlay))
#     reicmcl = np.zeros((ncol,nlay))
#     relqmcl = np.zeros((ncol,nlay))
#     taucmcl = np.zeros((ngptlw,ncol,nlay))

    # Preprocess the reic to >=10, otherwise the RRTMG will crash:
    #https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/rrtmg_lw_v4.85/gcm_model/src/rrtmg_lw_cldprmc.f90
    size=reic[0,:].size
    reic_temp=np.ones_like(play)
    for i in range(size):
        reic_temp[0,i]=max(reic[0,i],10)

    #  Call the Monte Carlo Independent Column Approximation (McICA, Pincus et al., JC, 2003)
    (cldfmcl, ciwpmcl, clwpmcl, reicmcl, relqmcl, taucmcl) = \
            rrtmg_lw.climlab_mcica_subcol_lw(
                        ncol, nlay, icld,
                        permuteseed, irng, play,
                        cldfrac, ciwp, clwp, reic_temp, relq, tauc)
    # Source code: https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/sourcemods/rrtmg_lw_rad.f90
    ispec=0
    (olr_sr, uflx, dflx, hr, uflxc, dflxc, hrc, duflx_dt, duflxc_dt) = \
                    rrtmg_lw.climlab_rrtmg_lw(ncol, nlay, icld, ispec, idrv,
                         play, plev, tlay, tlev, tsfc,
                         h2ovmr, o3vmr, co2vmr, ch4vmr, n2ovmr, o2vmr,
                         cfc11vmr,cfc12vmr, cfc22vmr, ccl4vmr, emis,
                         inflglw, iceflglw, liqflglw, cldfmcl,
                         taucmcl, ciwpmcl, clwpmcl, reicmcl, relqmcl,
                         tauaer)


    # Calculate the net longwave flux convergence
    up=uflx[0,:] # from surface to top
    down=dflx[0,:] # from surface to top
    num_lev=up.size # interface level numbers
    net_up=np.zeros(num_lev-1)
    net_down=np.zeros(num_lev-1)
    net=np.zeros(num_lev-1)
    for i in range(0,num_lev-1):
        # net upward LW fluxes
        #         In-out
        net_up[i]=up[i]-up[i+1] 
        # net downward LW fluxes
        #          In-out
        net_down[i]=down[i+1]-down[i]
        # net=netupward+netdownward
        net[i]=net_up[i]+net_down[i]

    lw_con=xr.DataArray(net[::-1], coords={"lev":np.array(mlev)}) 
    QRL_off=xr.DataArray(hr[0,::-1], coords={"lev":np.array(mlev)})
    LWUP  =xr.DataArray(up[::-1], coords={"ilev":np.array(ilev)})
    LWDN  =xr.DataArray(down[::-1], coords={"ilev":np.array(ilev)})
    NetLWUP=xr.DataArray(net_up[::-1], coords={"lev":np.array(mlev)}) 
    NetLWDN=xr.DataArray(net_down[::-1], coords={"lev":np.array(mlev)}) 
    # Calculate the unit transfer coefficient
    flux_to_hr=QRL_off/lw_con
    
    #=======================Do the shortwave calculation==============================================
    if coszen >0: # Day point
        nbndsw = int(rrtmg_sw.parrrsw.nbndsw)
        naerec = int(rrtmg_sw.parrrsw.naerec)
        ngptsw = int(rrtmg_sw.parrrsw.ngptsw)

        #  Initialize absorption data
        rrtmg_sw.climlab_rrtmg_sw_ini(cp)

        #  Lots of RRTMG parameters
        icld = 1    # Cloud overlap method, 0: Clear only, 1: Random, 2,  Maximum/random] 3: Maximum
        irng = 1  # more monte carlo stuff
        permuteseed = 150
        dyofyr = 0       # day of the year used to get Earth/Sun distance (if not adjes)
        inflgsw  = 2
        iceflgsw = 1
        liqflgsw = 1
        tauc = 0.
        ssac = 0.  # In-cloud single scattering albedo
        asmc = 0.  # In-cloud asymmetry parameter
        fsfc = 0.  # In-cloud forward scattering fraction (delta function pointing forward "forward peaked scattering")

        # AEROSOLS
        iaer = 0   #! Aerosol option flag
                    #!    0: No aerosol
                    #!    6: ECMWF method: use six ECMWF aerosol types input aerosol optical depth at 0.55 microns for each aerosol type (ecaer)
                    #!    10:Input aerosol optical properties: input total aerosol optical depth, single scattering albedo and asymmetry parameter (tauaer, ssaaer, asmaer) directly
        tauaer = 0. * np.ones_like(play)   # Aerosol optical depth (iaer=10 only), Dimensions,  (ncol,nlay,nbndsw)] #  (non-delta scaled)
        #  broadcast and transpose to get [ncol,nlay,nbndsw]
        tauaer = np.transpose(tauaer * np.ones([nbndsw,ncol,nlay]), (1,2,0))
        ssaaer = 0. * np.ones_like(play)   # Aerosol single scattering albedo (iaer=10 only), Dimensions,  (ncol,nlay,nbndsw)] #  (non-delta scaled)
        #  broadcast and transpose to get [ncol,nlay,nbndsw]
        ssaaer = np.transpose(ssaaer * np.ones([nbndsw,ncol,nlay]), (1,2,0))
        asmaer = 0. * np.ones_like(play)   # Aerosol asymmetry parameter (iaer=10 only), Dimensions,  (ncol,nlay,nbndsw)] #  (non-delta scaled)
        #  broadcast and transpose to get [ncol,nlay,nbndsw]
        asmaer = np.transpose(asmaer * np.ones([nbndsw,ncol,nlay]), (1,2,0))
        ecaer  = 0. * np.ones_like(play)   # Aerosol optical depth at 0.55 micron (iaer=6 only), Dimensions,  (ncol,nlay,naerec)] #  (non-delta scaled)
        #  broadcast and transpose to get [ncol,nlay,naerec]
        ecaer = np.transpose(ecaer * np.ones([naerec,ncol,nlay]), (1,2,0))

        #  These cloud arrays have an extra dimension for number of bands
        # in-cloud optical depth [nbndsw,ncol,nlay]
        tauc = 0. * np.ones_like(play)
        #  broadcast to get [nbndsw,ncol,nlay]
        tauc = tauc * np.ones([nbndsw,ncol,nlay])
        # In-cloud single scattering albedo, same operation
        ssac = 0. * np.ones_like(play) * np.ones([nbndsw,ncol,nlay])
        # In-cloud asymmetry parameter
        asmc = 0. * np.ones_like(play) * np.ones([nbndsw,ncol,nlay])
        # In-cloud forward scattering fraction (delta function pointing forward "forward peaked scattering")
        fsfc = 0. * np.ones_like(play) * np.ones([nbndsw,ncol,nlay])

        # insolation
        scon = 1365.2  # solar constant
        #     coszen = coszen  # cosine of zenith angle
        adjes = eccf  # instantaneous irradiance = scon * eccentricity_factor ## What is this? =>eccentricity factor
        dyofyr = 0       # day of the year used to get Earth/Sun distance (if not adjes)
        # new arguments for RRTMG_SW version 4.0
        isolvar = -1    # ! Flag for solar variability method
        indsolvar = np.ones(2)      # Facular and sunspot amplitude scale factors (isolvar=1),
                                     # or Mg and SB indices (isolvar=2)
        bndsolvar = np.ones(nbndsw) # Solar variability scale factors for each shortwave band
        solcycfrac = 1.              # Fraction of averaged solar cycle (0-1) at current time (isolvar=1)

        # Preprocess the reic-limit the reic to 13 to 130, otherwise RRTMG will crash:
        # https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_sw/rrtmg_sw_v4.0/gcm_model/src/rrtmg_sw_cldprmc.f90
        size=reic[0,:].size
        reic_temp=np.ones_like(play)
        for i in range(size):
            reic_temp[0,i]=min(max(reic[0,i],13),130)


        #  Call the Monte Carlo Independent Column Approximation (McICA, Pincus et al., JC, 2003)
        (cldfmcl, ciwpmcl, clwpmcl, reicmcl, relqmcl, taucmcl,
        ssacmcl, asmcmcl, fsfcmcl) = rrtmg_sw.climlab_mcica_subcol_sw(
                        ncol, nlay, icld, permuteseed, irng, play,
                        cldfrac, ciwp, clwp, reic_temp, relq, tauc, ssac, asmc, fsfc)

        (swuflx, swdflx, swhr, swuflxc, swdflxc, swhrc) = \
                rrtmg_sw.climlab_rrtmg_sw(ncol, nlay, icld, iaer,
                    play, plev, tlay, tlev, tsfc,
                    h2ovmr, o3vmr, co2vmr, ch4vmr, n2ovmr, o2vmr,
                    asdir, asdif, aldir, aldif,
                    coszen, adjes, dyofyr, scon, isolvar,
                    inflgsw, iceflgsw, liqflgsw, cldfmcl,
                    taucmcl, ssacmcl, asmcmcl, fsfcmcl,
                    ciwpmcl, clwpmcl, reicmcl, relqmcl,
                    tauaer, ssaaer, asmaer, ecaer,
                    bndsolvar, indsolvar, solcycfrac)

        # Calculate the net radiation flux
        up=swuflx[0,:] # from surface to top
        down=swdflx[0,:] # from surface to top
        num_lev=up.size # interface level numbers
        net_up=np.zeros(num_lev-1)
        net_down=np.zeros(num_lev-1)
        net=np.zeros(num_lev-1)
        for i in range(0,num_lev-1):
            # net upward LW fluxes
            #         In-out
            net_up[i]=up[i]-up[i+1] 
            # net downward LW fluxes
            #          In-out
            net_down[i]=down[i+1]-down[i]
            # net=netupward+netdownward
            net[i]=net_up[i]+net_down[i]

        sw_con=xr.DataArray(net[::-1], coords={"lev":np.array(mlev)}) 
        SWUP  =xr.DataArray(up[::-1], coords={"ilev":np.array(ilev)})
        SWDN  =xr.DataArray(down[::-1], coords={"ilev":np.array(ilev)})
        NetSWUP=xr.DataArray(net_up[::-1], coords={"lev":np.array(mlev)}) 
        NetSWDN=xr.DataArray(net_down[::-1], coords={"lev":np.array(mlev)})
        QRS_off=sw_con*flux_to_hr
        
    else: # Night point
        ## Is this right? Is LWUP affected?
        SWUP   =LWUP.copy()*0.0
        SWDN   =LWDN.copy()*0.0
        NetSWUP=NetLWUP.copy()*0.0
        NetSWDN=NetLWDN.copy()*0.0
        QRS_off=QRL_off.copy()*0.0
    
    return QRL_off,QRS_off,LWUP,LWDN,NetLWUP,NetLWDN,SWUP,SWDN,NetSWUP,NetSWDN

In [ ]:
def offline_LW(nlay,plev,play,tsfc,tlev,tlay,h2ovmr,o3vmr,co2vmr,ch4vmr,n2ovmr,cfc11vmr,cfc12vmr,o2vmr,cfc22vmr,ccl4vmr,\
           cldfrac,ciwp, clwp, reic, relq, mlev,ilev):
    
    # Specific heat at constant pressure
    cp = 1004.
    ncol = 1

    #====================================Do the longwave calculation==============================================
    # nbndlw and ngptlw
    nbndlw=rrtmg_lw.parrrtm.nbndlw
    ngptlw=rrtmg_lw.parrrtm.ngptlw 
    
    ## First, initialize the data
    rrtmg_lw.climlab_rrtmg_lw_ini(cp)

    ## Lots of RRTMG parameters
    icld = 1    # Cloud overlap method, 0: Clear only, 1: Random, 2,  Maximum/random] 3: Maximum
    irng = 1  # more monte carlo stuff
    idrv = 0  # whether to also calculate the derivative of flux with respect to surface temp
    permuteseed = 300
    inflglw  = 2
    # Set all these variables to 0, same as CCM3:
    # https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/rrtmg_lw_v4.85/gcm_model/src/rrtmg_lw_cldprmc.f90
    iceflglw = 0
    liqflglw = 0

   
    #  These arrays have an extra dimension for number of bands
    # in-cloud optical depth [nbndlw,ncol,nlay]
    tauc = 0. * np.ones_like(play)
    #  broadcast to get [nbndlw,ncol,nlay]
    tauc = tauc * np.ones([nbndlw,ncol,nlay])
    # Aerosol optical depth at mid-point of LW spectral bands [ncol,nlay,nbndlw]
    tauaer = 0. * np.ones_like(play)
    #  broadcast and transpose to get [ncol,nlay,nbndlw]
    tauaer = np.transpose(tauaer * np.ones([nbndlw,ncol,nlay]), (1,2,0))

    # surface emissivity
    emis = 1. * np.ones((ncol,nbndlw))

#     # Clear-sky only first
#     cldfmcl = np.zeros((ngptlw,ncol,nlay))
#     ciwpmcl = np.zeros((ngptlw,ncol,nlay))
#     clwpmcl = np.zeros((ngptlw,ncol,nlay))
#     reicmcl = np.zeros((ncol,nlay))
#     relqmcl = np.zeros((ncol,nlay))
#     taucmcl = np.zeros((ngptlw,ncol,nlay))

    # Preprocess the reic to >=10, otherwise the RRTMG will crash:
    #https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/rrtmg_lw_v4.85/gcm_model/src/rrtmg_lw_cldprmc.f90
    size=reic[0,:].size
    reic_temp=np.ones_like(play)
    for i in range(size):
        reic_temp[0,i]=max(reic[0,i],10)

    #  Call the Monte Carlo Independent Column Approximation (McICA, Pincus et al., JC, 2003)
    (cldfmcl, ciwpmcl, clwpmcl, reicmcl, relqmcl, taucmcl) = \
            rrtmg_lw.climlab_mcica_subcol_lw(
                        ncol, nlay, icld,
                        permuteseed, irng, play,
                        cldfrac, ciwp, clwp, reic_temp, relq, tauc)
    # Source code: https://github.com/climlab/climlab-rrtmg/blob/main/climlab_rrtmg/rrtmg_lw/sourcemods/rrtmg_lw_rad.f90
    ispec=0
    (olr_sr, uflx, dflx, hr, uflxc, dflxc, hrc, duflx_dt, duflxc_dt) = \
                    rrtmg_lw.climlab_rrtmg_lw(ncol, nlay, icld, ispec, idrv,
                         play, plev, tlay, tlev, tsfc,
                         h2ovmr, o3vmr, co2vmr, ch4vmr, n2ovmr, o2vmr,
                         cfc11vmr,cfc12vmr, cfc22vmr, ccl4vmr, emis,
                         inflglw, iceflglw, liqflglw, cldfmcl,
                         taucmcl, ciwpmcl, clwpmcl, reicmcl, relqmcl,
                         tauaer)


    # Calculate the net longwave flux convergence
    up=uflx[0,:] # from surface to top
    down=dflx[0,:] # from surface to top
    num_lev=up.size # interface level numbers
    net_up=np.zeros(num_lev-1)
    net_down=np.zeros(num_lev-1)
    net=np.zeros(num_lev-1)
    for i in range(0,num_lev-1):
        # net upward LW fluxes
        #         In-out
        net_up[i]=up[i]-up[i+1] 
        # net downward LW fluxes
        #          In-out
        net_down[i]=down[i+1]-down[i]
        # net=netupward+netdownward
        net[i]=net_up[i]+net_down[i]

    lw_con=xr.DataArray(net[::-1], coords={"lev":np.array(mlev)}) 
    QRL_off=xr.DataArray(hr[0,::-1], coords={"lev":np.array(mlev)}) #what is the unit here?
    LWUP  =xr.DataArray(up[::-1], coords={"ilev":np.array(ilev)})
    LWDN  =xr.DataArray(down[::-1], coords={"ilev":np.array(ilev)})
    NetLWUP=xr.DataArray(net_up[::-1], coords={"lev":np.array(mlev)}) 
    NetLWDN=xr.DataArray(net_down[::-1], coords={"lev":np.array(mlev)}) 
    # Calculate the unit transfer coefficient
    flux_to_hr=QRL_off/lw_con
    
    return QRL_off,LWUP,LWDN,NetLWUP,NetLWDN